In [1]:
"""
CS 484 Final Project

Alexander Wei, a6wei@uwaterloo.ca, 20836214
Kaiz Nanji

Project #5
Semi-supervised image classification: assuming that only M out of N images in the traingin data have ground truth labels,
design and implement a weakly supervsied training of classification network that can benefit from unlabeled examples in
the training dataset(e.g. MNIST or CIFAR-10, but you need to ignore labels on a subset of training examples).You should
demonstrate how the performance changes as M gets progressively smaller. While you can use any well-motivated ideas, one
basic approach could be to combine cross-entropy on labeled points with (unsupervised) K-means clustering loss over deep
features (e.g. in the last layer before the linear classifier). It is also advisable to use augmentation (a loss
enforcing consistent labeling of augmented training examples). You can also explore Mutual Information loss function
formulated in Bridle & MacKay "Unsupervised Classifiers, Mutual Information and Phantom Targets", NIPS 1991. 
"""

'\nCS 484 Final Project\n\nAlexander Wei, a6wei@uwaterloo.ca, 20836214\nKaiz Nanji\n\nProject #5\nSemi-supervised image classification: assuming that only M out of N images in the traingin data have ground truth labels,\ndesign and implement a weakly supervsied training of classification network that can benefit from unlabeled examples in\nthe training dataset(e.g. MNIST or CIFAR-10, but you need to ignore labels on a subset of training examples).You should\ndemonstrate how the performance changes as M gets progressively smaller. While you can use any well-motivated ideas, one\nbasic approach could be to combine cross-entropy on labeled points with (unsupervised) K-means clustering loss over deep\nfeatures (e.g. in the last layer before the linear classifier). It is also advisable to use augmentation (a loss\nenforcing consistent labeling of augmented training examples). You can also explore Mutual Information loss function\nformulated in Bridle & MacKay "Unsupervised Classifiers, Mutu

In [2]:
from src.model import *
from src.utils import *

import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:
print(torch.cuda.device_count())

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

1
cuda


In [4]:
"""
Checks the filepath "./data/MNIST/raw" for the dataset. If not found, downloads the dataset
"""
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,), (0.3081,))
])

mnist_train, mnist_test = download(transform)


In [5]:
# Increase TRAIN_BATCH_SIZE if you are using GPU to speed up training. 
# When batch size changes, the learning rate may also need to be adjusted. 
# Note that batch size maybe limited by your GPU memory, so adjust if you get "run out of GPU memory" error.
TRAIN_BATCH_SIZE = 100

# If you are NOT using Windows, set NUM_WORKERS to anything you want, e.g. NUM_WORKERS = 4,
# but Windows has issues with multi-process dataloaders, so NUM_WORKERS must be 0 for Windows.
NUM_WORKERS = 0


train_loader = DataLoader(mnist_train, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
val_loader = DataLoader(mnist_test, batch_size=1, num_workers=NUM_WORKERS, shuffle=False)

In [6]:
my_gmm = MyGaussianMixture()
my_gmm.clear()

my_loss_function = nn.CrossEntropyLoss()
my_loss_function = my_loss_function.to(device)

my_model = MyModel(device, my_gmm, my_loss_function)
my_model = my_model.to(device)

sgd_optimizer = get_optimizer(my_model)

In [7]:
loss_graph = []

loss = train_model(my_model, device, train_loader, sgd_optimizer, loss_graph)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
%matplotlib inline

plt.clf()
plt.cla()
plt.plot(loss_graph)
plt.ylabel("Cross Entropy Loss")
plt.xlabel("Batch Number")
plt.title("Training Loss Over Batch Number")
plt.show()


In [ ]:
val_loss, accurate, confusion_matrix = validate_model(my_model, device, val_loader)

In [ ]:
print(val_loss, accurate)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=[i for i in range(10)])
disp.plot()
plt.show()